
# *Code for the Paper titled " Design of Shark Detection and Decoy Buoys "*
---

Find the Preprint verison of the paper at : https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4326417

---
Authors : Abhishek Sebastian , Pragna Rakesh , Geetha K S and Aswin Rajendran

---

If you find the dataset and code useful :

> Cite: Sebastian, Abhishek and R, Pragna and K S, Geetha and R, Aswin, Design of Shark Detection and Decoy Buoys (January 4, 2023). Proceedings of the International Conference on Intelligent Data Communication Technologies and Internet of Things (IDCIoT 2023),662-669.

Acknowledgement: Team Ratacaya (Nationals - SIH '2022 )


# Step 1: Clone YOLOv5 Module


In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5

%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 14441, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 14441 (delta 42), reused 44 (delta 22), pack-reused 14371
Receiving objects: 100% (14441/14441), 13.57 MiB | 23.43 MiB/s, done.
Resolving deltas: 100% (9943/9943), done.
/content/yolov5
     |████████████████████████████████| 182 kB 9.1 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 1.6 MB 25.9 MB/s 
     |████████████████████████████████| 42 kB 910 kB/s 
     |████████████████████████████████| 138 kB 29.4 MB/s 
     |████████████████████████████████| 145 kB 63.5 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
     |████████████████████████████████| 178 kB 79.6 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
Setup complete. Using torch 1.13.0+cu116 (Tesla T4)


# **Step 2: Preparing Data Set For Object Detection :**



In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="81ubU64frjed11AQcaqJ")
project = rf.workspace("abhishek-sebastian").project("sdb")
dataset = project.version(2).download("yolov5")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
loading Roboflow workspace...
loading Roboflow project...

Extracting Dataset Version Zip to SDB-2 in yolov5pytorch:: 100%|██████████| 4390/4390 [00:02<00:00, 1979.81it/s]


# Step 3: Train Our Custom YOLOv5 model

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset locaiton is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training

In [ ]:
!python train.py --img 416 --batch 16 --epochs 50 --data {dataset.location}/data.yaml --weights yolov5x.pt --cache

train: weights=yolov5x.pt, cfg=, data=/content/yolov5/SDB-2/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-30-g342fe05 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_

# Evaluate Custom YOLOv5 Detector Performance
Training losses and performance metrics are saved to Tensorboard and also to a logfile.

If you are new to these metrics, the one you want to focus on is `mAP_0.5` - learn more about mean average precision [here](https://blog.roboflow.com/mean-average-precision/).

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

<IPython.core.display.Javascript object>

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
!python detect.py --weights "./runs/train/exp/weights/best.pt" --img 640 --conf 0.1 --source {dataset.location}/test/images

detect: weights=['./runs/train/exp/weights/best.pt'], source=/content/yolov5/SDB-2/test/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-30-g342fe05 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 322 layers, 86274349 parameters, 0 gradients, 204.1 GFLOPs
image 1/216 /content/yolov5/SDB-2/test/images/Basking_13_jpeg.rf.32e656782605862fabd37c7569acb862.jpg: 384x640 1 Basking Shark, 48.4ms
image 2/216 /content/yolov5/SDB-2/test/images/Basking_24_jpeg.rf.741338ab140f1bcc4afe43f470dc3bc4.jpg: 416x640 (no detections), 49.4ms
image 3/216 /content/yolov5/SDB-2/test/images/Basking_2_jpeg

In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp3/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

In [ ]:
!python export.py --weights "/content/drive/MyDrive/Sharks/best.pt"  --include tflite

export: data=data/coco128.yaml, weights=['/content/drive/MyDrive/Sharks/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['tflite']
YOLOv5 🚀 v7.0-30-g342fe05 Python-3.8.16 torch-1.13.0+cu116 CPU

Traceback (most recent call last):
  File "export.py", line 653, in <module>
    main(opt)
  File "export.py", line 648, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.8/dist-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "export.py", line 528, in run
    model = attempt_load(weights, device=device, inplace=True, fuse=True)  # load FP32 model
  File "/content/yolov5/models/experimental.py", line 79, in attempt_load
    ckpt = torch.load(attempt_download(w), map_location='cpu')

# Conclusion and Next Steps

Congratulations! You've trained a custom YOLOv5 model to recognize Shark Species.


In [ ]:
#export your model's weights for future use
from google.colab import files
files.download('./runs/train/exp/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

A way to alert ( WEB CAMERA )

In [ ]:
import cv2
import yolov5
import webbrowser

# Load a pretrained YOLOv5 model
model = yolov5.load_model("yolov5s.yaml", "weights/best.pt")

# Define a user-defined function
def custom_function(x, class_id):
    if class_id in [2,3,5]:  
        webbrowser.open("https://www.example.com")
    return x

# Open a connection to the camera
cap = cv2.VideoCapture(0)

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()

    # Run the frame through the model
    output, _, class_ids = model(frame, classes=[2, 3,5], custom_function=custom_function)

    # Show the output on the screen
    cv2.imshow("Object Detection", output)

    # Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the window
cap.release()
cv2.destroyAllWindows()


A way to alert (youtube video)


In [ ]:
import cv2
import yolov5
from pytube import YouTube

# Download the YouTube video
yt = YouTube("https://www.youtube.com/watch?v=video_id")
video = yt.streams.first()
video.download("path/to/save/video")

# Load a pretrained YOLOv5 model
model = yolov5.load_model("yolov5s.yaml", "weights/best.pt")

# Define a user-defined function
def custom_function(x, class_id):
    if class_id == 2:  # class_id is the ID of the detected object
        webbrowser.open("https://www.example.com")
    return x

# Open the video using OpenCV
cap = cv2.VideoCapture("path/to/save/video")

while True:
    # Capture a frame from the video
    ret, frame = cap.read()

    # Run the frame through the model
    output, _, class_ids = model(frame, classes=[2, 3], custom_function=custom_function)

    # Show the output on the screen
    cv2.imshow("Object Detection", output)

    # Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video and close the window
cap.release()
cv2.destroyAllWindows()
